Importation des librairies

In [1]:
import numpy as np
import cv2
from tensorflow import keras
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [2]:
threshold = 0.75  # THRESHOLD
font = cv2.FONT_HERSHEY_SIMPLEX
model = keras.models.load_model('./traffic_sign_model.h5') #load model

In [3]:
def getClassName(classNo):
    if classNo == 0:
        return 'Speed Limit 20 km/h'
    elif classNo == 1:
        return 'Speed Limit 30 km/h'
    elif classNo == 2:
        return 'Speed Limit 50 km/h'
    elif classNo == 3:
        return 'Speed Limit 60 km/h'
    elif classNo == 4:
        return 'Speed Limit 70 km/h'
    elif classNo == 5:
        return 'Speed Limit 80 km/h'
    elif classNo == 6:
        return 'End of Speed Limit 80 km/h'
    elif classNo == 7:
        return 'Speed Limit 100 km/h'
    elif classNo == 8:
        return 'Speed Limit 120 km/h'
    elif classNo == 9:
        return 'No passing'
    elif classNo == 10:
        return 'No passing for vechiles over 3.5 metric tons'
    elif classNo == 11:
        return 'Right-of-way at the next intersection'
    elif classNo == 12:
        return 'Priority road'
    elif classNo == 13:
        return 'Yield'
    elif classNo == 14:
        return 'Stop'
    elif classNo == 15:
        return 'No vechiles'
    elif classNo == 16:
        return 'Vechiles over 3.5 metric tons prohibited'
    elif classNo == 17:
        return 'No entry'
    elif classNo == 18:
        return 'General caution'
    elif classNo == 19:
        return 'Dangerous curve to the left'
    elif classNo == 20:
        return 'Dangerous curve to the right'
    elif classNo == 21:
        return 'Double curve'
    elif classNo == 22:
        return 'Bumpy road'
    elif classNo == 23:
        return 'Slippery road'
    elif classNo == 24:
        return 'Road narrows on the right'
    elif classNo == 25:
        return 'Road work'
    elif classNo == 26:
        return 'Traffic signals'
    elif classNo == 27:
        return 'Pedestrians'
    elif classNo == 28:
        return 'Children crossing'
    elif classNo == 29:
        return 'Bicycles crossing'
    elif classNo == 30:
        return 'Beware of ice/snow'
    elif classNo == 31:
        return 'Wild animals crossing'
    elif classNo == 32:
        return 'End of all speed and passing limits'
    elif classNo == 33:
        return 'Turn right ahead'
    elif classNo == 34:
        return 'Turn left ahead'
    elif classNo == 35:
        return 'Ahead only'
    elif classNo == 36:
        return 'Go straight or right'
    elif classNo == 37:
        return 'Go straight or left'
    elif classNo == 38:
        return 'Keep right'
    elif classNo == 39:
        return 'Keep left'
    elif classNo == 40:
        return 'Roundabout mandatory'
    elif classNo == 41:
        return 'End of no passing'
    elif classNo == 42:
        return 'End of no passing by vechiles over 3.5 metric tons'

In [4]:
#Fonction de pretraitement des images 

def preprocess_img(imgBGR, erode_dilate=True):  # pre-processing fro detect signs in  image.
    rows, cols, _ = imgBGR.shape
    imgHSV = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2HSV)
    Bmin = np.array([100, 43, 46])
    Bmax = np.array([124, 255, 255])
    img_Bbin = cv2.inRange(imgHSV, Bmin, Bmax)

    Rmin1 = np.array([0, 43, 46])
    Rmax1 = np.array([10, 255, 255])
    img_Rbin1 = cv2.inRange(imgHSV, Rmin1, Rmax1)

    Rmin2 = np.array([156, 43, 46])
    Rmax2 = np.array([180, 255, 255])
    img_Rbin2 = cv2.inRange(imgHSV, Rmin2, Rmax2)
    img_Rbin = np.maximum(img_Rbin1, img_Rbin2)
    img_bin = np.maximum(img_Bbin, img_Rbin)

    if erode_dilate is True:
        kernelErosion = np.ones((3, 3), np.uint8)
        kernelDilation = np.ones((3, 3), np.uint8)
        img_bin = cv2.erode(img_bin, kernelErosion, iterations=2)
        img_bin = cv2.dilate(img_bin, kernelDilation, iterations=2)

    return img_bin

In [5]:
def contour_detect(img_bin, min_area, max_area=-1, wh_ratio=2.0):
    rects = []
    contours, _ = cv2.findContours(img_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    if len(contours) == 0:
        return rects

    max_area = img_bin.shape[0] * img_bin.shape[1] if max_area < 0 else max_area
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= min_area and area <= max_area:
            x, y, w, h = cv2.boundingRect(contour)
            if 1.0 * w / h < wh_ratio and 1.0 * h / w < wh_ratio:
                rects.append([x, y, w, h])
    return rects


"""def contour_detect(img_bin, min_area, max_area=-1, wh_ratio=2.0, aspect_ratio_threshold=0.4):
    rects = []
    contours, _ = cv2.findContours(img_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    if len(contours) == 0:
        return rects

    max_area = img_bin.shape[0] * img_bin.shape[1] if max_area < 0 else max_area
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >= min_area and area <= max_area:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = 1.0 * w / h
            if aspect_ratio_threshold * wh_ratio <= aspect_ratio <= wh_ratio / aspect_ratio_threshold:
                rects.append([x, y, w, h])
    return rects
    """


'def contour_detect(img_bin, min_area, max_area=-1, wh_ratio=2.0, aspect_ratio_threshold=0.4):\n    rects = []\n    contours, _ = cv2.findContours(img_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) \n    if len(contours) == 0:\n        return rects\n\n    max_area = img_bin.shape[0] * img_bin.shape[1] if max_area < 0 else max_area\n    for contour in contours:\n        area = cv2.contourArea(contour)\n        if area >= min_area and area <= max_area:\n            x, y, w, h = cv2.boundingRect(contour)\n            aspect_ratio = 1.0 * w / h\n            if aspect_ratio_threshold * wh_ratio <= aspect_ratio <= wh_ratio / aspect_ratio_threshold:\n                rects.append([x, y, w, h])\n    return rects\n    '

In [6]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

Fonction principale pour lire la video et detecter le panneau dessus

In [7]:
if __name__ == "__main__":
    video_path = '../../../files/video.mp4'
    cap = cv2.VideoCapture(video_path)
    cols = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    rows = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Video", cols, rows)
    

    while (1):
        ret, img = cap.read()
        img_bin = preprocess_img(img, False);cv2.imshow("bin image", img_bin)
        min_area = img_bin.shape[0] * img.shape[1] / (25 * 25)
        rects = contour_detect(img_bin, min_area=min_area)   # get x,y,h and w.
        img_bbx = img.copy()
        
        for rect in rects:
            xc = int(rect[0] + rect[2] / 2)
            yc = int(rect[1] + rect[3] / 2)

            size = max(rect[2], rect[3])
            x1 = max(0, int(xc - size / 2))
            y1 = max(0, int(yc - size / 2))
            x2 = min(cols, int(xc + size / 2))
            y2 = min(rows, int(yc + size / 2))

            # rect[2] is width and rect[3] for height
            if rect[2] > 100 and rect[3] > 100:             #only detect those signs whose height and width >100
                cv2.rectangle(img_bbx, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 0, 255), 2)
            crop_img = np.asarray(img[y1:y2, x1:x2])
            crop_img = cv2.resize(crop_img, (32, 32))
            crop_img = preprocessing(crop_img)
            cv2.imshow("afterprocessing", crop_img)
            crop_img = crop_img.reshape(1, 32, 32, 1)       # (1,32,32) after reshape it become (1,32,32,1)
            predictions = model.predict(crop_img)           # make predicion
            classIndex = np.argmax(predictions)
            probabilityValue = np.amax(predictions)
            if probabilityValue > threshold:
                #write class name on the output screen
                speed_limit = cv2.putText(img_bbx, str(classIndex) + " " + str(getClassName(classIndex)), (rect[0], rect[1] - 10),
                            font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
                # write probability value on the output screen
                cv2.putText(img_bbx, str(round(probabilityValue * 100, 2)) + "%", (rect[0], rect[1] - 40), font, 0.75,
                            (0, 0, 255), 2, cv2.LINE_AA)

        cv2.imshow("detect result", img_bbx)
        if cv2.waitKey(1) & 0xFF == ord('q'):           # q for quit 
            break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step


Extraction des informations gps sur la video

In [13]:
import cv2
import piexif
from PIL import Image

def extract_gps_from_video(video_path):
    # Ouvrir la vidéo à l'aide d'OpenCV
    cap = cv2.VideoCapture(video_path)

    # Liste pour stocker les coordonnées GPS
    gps_coordinates = []

    # Lire chaque frame de la vidéo
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir la frame en format PIL Image pour extraire les métadonnées EXIF
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Extraire les métadonnées EXIF
        exif_data = pil_image.info.get('exif')

        if exif_data:
            # Vérifier si les métadonnées GPS sont présentes
            if piexif.GPSIFD.GPSLatitude in exif_data and piexif.GPSIFD.GPSLongitude in exif_data:
                latitude = exif_data[piexif.GPSIFD.GPSLatitude]
                longitude = exif_data[piexif.GPSIFD.GPSLongitude]

                # Convertir les coordonnées GPS en degrés décimaux
                latitude_decimal = latitude[0][0] / latitude[0][1] + latitude[1][0] / latitude[1][1] / 60 + latitude[2][0] / latitude[2][1] / 3600
                longitude_decimal = longitude[0][0] / longitude[0][1] + longitude[1][0] / longitude[1][1] / 60 + longitude[2][0] / longitude[2][1] / 3600

                # Ajouter les coordonnées GPS à la liste
                gps_coordinates.append((latitude_decimal, longitude_decimal))

    # Fermer la capture vidéo
    cap.release()

    return gps_coordinates

# Exemple d'utilisation
video_path = "../../files/vid4.mp4"
gps_coordinates = extract_gps_from_video(video_path)

if gps_coordinates:
    for i, (latitude, longitude) in enumerate(gps_coordinates):
        print(f"Frame {i+1}: Latitude: {latitude}, Longitude: {longitude}")
else:
    print("Aucune information GPS trouvée dans la vidéo.")


Aucune information GPS trouvée dans la vidéo.


In [7]:
def get_gps_info(photo_path):
    try:
        img = Image.open(photo_path)
        exif_data = img._getexif()
        if exif_data is not None:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                if tag_name == "GPSInfo":
                    return value
    except (AttributeError, KeyError, IndexError):
        pass
    return None

def decimal_to_dms(value):
    degrees = int(value)
    minutes = int((value - degrees) * 60)
    seconds = int((value - degrees - minutes / 60) * 3600)
    return degrees, minutes, seconds

Extraction des donnees gps sur l'image

In [8]:
def get_geolocation(exif_data):
    # Coordonnées par défaut (au cas où les données GPS ne sont pas présentes)
    latitude = None
    longitude = None

    # Vérifier si les données GPS sont présentes dans les métadonnées EXIF
    if 'GPSInfo' in exif_data:
        # Convertir les valeurs degré/minute/seconde en degrés décimaux
        def convert_to_degrees(value):
            d = value[0]
            m = value[1]
            s = value[2]
            return d + (m / 60.0) + (s / 3600.0)

        # Extraire les informations GPS
        gps_info = exif_data['GPSInfo']
        gps_latitude = gps_info.get(2)
        gps_latitude_ref = gps_info.get(1)
        gps_longitude = gps_info.get(4)
        gps_longitude_ref = gps_info.get(3)

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            latitude = convert_to_degrees(gps_latitude)
            if gps_latitude_ref != 'N':
                latitude = -latitude

            longitude = convert_to_degrees(gps_longitude)
            if gps_longitude_ref != 'E':
                longitude = -longitude

    return latitude, longitude

def extract_geolocation_from_photo(photo_path):
    # Ouvrir l'image avec PIL
    image = Image.open(photo_path)

    # Extraire les métadonnées EXIF
    exif_data = image._getexif()

    # Si les métadonnées EXIF existent, les convertir en un dictionnaire avec des noms explicites
    if exif_data is not None:
        exif_data = {
            TAGS[key]: exif_data[key]
            for key in exif_data
            if key in TAGS
        }

    # Obtenir les coordonnées GPS de l'image
    latitude, longitude = get_geolocation(exif_data)

    return latitude, longitude

# Exemple d'utilisation
photo_path = "../../file3.jpg"
latitude, longitude = extract_geolocation_from_photo(photo_path)

if latitude is not None and longitude is not None:
    print(f"Latitude: {latitude}")
    print(f"Longitude: {longitude}")
else:
    print("Aucune information de géolocalisation trouvée dans la photo.")


Latitude: 4.069917194444444
Longitude: 9.726550083333333


Fonction principale pour lire l'image, detecter le panneau dessus et renvoyer les informations gps s'il y en a

In [9]:
if __name__ == "__main__":
    photo_path = '../../files/IMG_20230805_112320_984.jpg'  # Replace this with the path to your photo
    img = cv2.imread(photo_path)

    img = cv2.resize(img, (550, 600), interpolation=cv2.INTER_LINEAR)  # Resize the original image

    img_bin = preprocess_img(img, False)
    min_area = img_bin.shape[0] * img.shape[1] / (25 * 25)
    rects = contour_detect(img_bin, min_area=min_area)
    img_bbx = img.copy()

    gps_info = get_gps_info(photo_path)
    
    if gps_info is not None:
        latitude = gps_info[2][0] + gps_info[2][1] / 60 + gps_info[2][2] / 3600
        longitude = gps_info[4][0] + gps_info[4][1] / 60 + gps_info[4][2] / 3600
        lat_deg, lat_min, lat_sec = decimal_to_dms(latitude)
        lon_deg, lon_min, lon_sec = decimal_to_dms(longitude)

        print("GPS Coordinates:")
        print(f"Latitude: {lat_deg}° {lat_min}' {lat_sec}\"")
        print(f"Longitude: {lon_deg}° {lon_min}' {lon_sec}\"")
        
    else: 
        print('No gps information')

    for rect in rects:
        xc = int(rect[0] + rect[2] / 2)
        yc = int(rect[1] + rect[3] / 2)

        size = max(rect[2], rect[3])
        x1 = max(0, int(xc - size / 2))
        y1 = max(0, int(yc - size / 2))
        x2 = min(img.shape[1], int(xc + size / 2))
        y2 = min(img.shape[0], int(yc + size / 2))

        if rect[2] > 100 and rect[3] > 100:
            cv2.rectangle(img_bbx, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 0, 255), 2)

        crop_img = np.asarray(img[y1:y2, x1:x2])
        crop_img = cv2.resize(crop_img, (32, 32))
        crop_img = preprocessing(crop_img)
        crop_img = crop_img.reshape(1, 32, 32, 1)

        # Assume 'model' and 'getClassName' are properly defined for image recognition
        predictions = model.predict(crop_img)
        classIndex = np.argmax(predictions)
        probabilityValue = np.amax(predictions)
        threshold = 0.5  # You can adjust the threshold value as needed

        if probabilityValue > threshold:
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img_bbx, str(classIndex) + " " + str(getClassName(classIndex)),
                        (rect[0], rect[1] - 10), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

            cv2.putText(img_bbx, str(round(probabilityValue * 100, 2)) + "%",
                        (rect[0], rect[1] - 40), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow("detect result", img_bbx)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


GPS Coordinates:
Latitude: 4° 4' 44"
Longitude: 9° 47' 25"
1/1 [==============================] - 0s 32ms/step


In [10]:
import tkinter as tk
from tkinter import filedialog


def detect_panels_on_image():
    global img_path
    photo_path = filedialog.askopenfilename(initialdir="/", title="Select an Image", filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif")])
    if photo_path:
        img = cv2.imread(photo_path)

        img = cv2.resize(img, (550, 600), interpolation=cv2.INTER_LINEAR)

        img_bin = preprocess_img(img, False)
        min_area = img_bin.shape[0] * img.shape[1] / (25 * 25)
        rects = contour_detect(img_bin, min_area)
        img_bbx = img.copy()

        gps_info = get_gps_info(photo_path)
        
        if gps_info is not None:
            latitude = gps_info[2][0] + gps_info[2][1] / 60 + gps_info[2][2] / 3600
            longitude = gps_info[4][0] + gps_info[4][1] / 60 + gps_info[4][2] / 3600
            lat_deg, lat_min, lat_sec = decimal_to_dms(latitude)
            lon_deg, lon_min, lon_sec = decimal_to_dms(longitude)

            print("GPS Coordinates:")
            print(f"Latitude: {lat_deg}° {lat_min}' {lat_sec}\"")
            print(f"Longitude: {lon_deg}° {lon_min}' {lon_sec}\"")
            
        else: 
            print('No gps information')

        for rect in rects:
            xc = int(rect[0] + rect[2] / 2)
            yc = int(rect[1] + rect[3] / 2)

            size = max(rect[2], rect[3])
            x1 = max(0, int(xc - size / 2))
            y1 = max(0, int(yc - size / 2))
            x2 = min(img.shape[1], int(xc + size / 2))
            y2 = min(img.shape[0], int(yc + size / 2))

            if rect[2] > 100 and rect[3] > 100:
                cv2.rectangle(img_bbx, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 0, 255), 2)

            crop_img = np.asarray(img[y1:y2, x1:x2])
            crop_img = cv2.resize(crop_img, (32, 32))
            crop_img = preprocessing(crop_img)
            crop_img = crop_img.reshape(1, 32, 32, 1)

            # Assume 'model' and 'getClassName' are properly defined for image recognition
            predictions = model.predict(crop_img)
            classIndex = np.argmax(predictions)
            probabilityValue = np.amax(predictions)
            threshold = 0.5

            if probabilityValue > threshold:
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img_bbx, str(classIndex) + " " + str(getClassName(classIndex)),
                            (rect[0], rect[1] - 10), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

                cv2.putText(img_bbx, str(round(probabilityValue * 100, 2)) + "%",
                            (rect[0], rect[1] - 40), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

        cv2.imshow("detect result", img_bbx)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
def detect_panels_on_video():
    video_path = filedialog.askopenfilename(initialdir="/", title="Select a Video", filetypes=[("Video files", "*.mp4;*.avi")])
    if video_path:
        video_capture = cv2.VideoCapture(video_path)

        while True:
            ret, frame = video_capture.read()

            if not ret:
                break

            frame = cv2.resize(frame, (550, 600), interpolation=cv2.INTER_LINEAR)

            img_bin = preprocess_img(frame, False)
            min_area = img_bin.shape[0] * frame.shape[1] / (25 * 25)
            rects = contour_detect(img_bin, min_area)
            img_bbx = frame.copy()

            #gps_info = get_gps_info(photo_path)
        
            """if extract_gps_from_video is not None:
                latitude = extract_gps_from_video[2][0] + extract_gps_from_video[2][1] / 60 + extract_gps_from_video[2][2] / 3600
                longitude = extract_gps_from_video[4][0] + extract_gps_from_video[4][1] / 60 + extract_gps_from_video[4][2] / 3600
                lat_deg, lat_min, lat_sec = decimal_to_dms(latitude)
                lon_deg, lon_min, lon_sec = decimal_to_dms(longitude)

                print("GPS Coordinates:")
                print(f"Latitude: {lat_deg}° {lat_min}' {lat_sec}\"")
                print(f"Longitude: {lon_deg}° {lon_min}' {lon_sec}\"")
                
            else: 
                print('No gps information')
                """
                
            for rect in rects:
                xc = int(rect[0] + rect[2] / 2)
                yc = int(rect[1] + rect[3] / 2)

                size = max(rect[2], rect[3])
                x1 = max(0, int(xc - size / 2))
                y1 = max(0, int(yc - size / 2))
                x2 = min(frame.shape[1], int(xc + size / 2))#
                y2 = min(frame.shape[0], int(yc + size / 2))#

                if rect[2] > 100 and rect[3] > 100:
                    cv2.rectangle(img_bbx, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 0, 255), 2)

                crop_img = np.asarray(frame[y1:y2, x1:x2])#
                crop_img = cv2.resize(crop_img, (32, 32))
                crop_img = preprocessing(crop_img)
                crop_img = crop_img.reshape(1, 32, 32, 1)

                # Assume 'model' and 'getClassName' are properly defined for image recognition
                predictions = model.predict(crop_img)
                classIndex = np.argmax(predictions)
                probabilityValue = np.amax(predictions)
                threshold = 0.5

            if probabilityValue > threshold:
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img_bbx, str(classIndex) + " " + str(getClassName(classIndex)),
                            (rect[0], rect[1] - 10), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

                cv2.putText(img_bbx, str(round(probabilityValue * 100, 2)) + "%",
                            (rect[0], rect[1] - 40), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

            cv2.imshow("detect result", img_bbx)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()

# Create the main application window
root = tk.Tk()
root.title("Image Panel Detection")
root.geometry('300x100')
root['bg'] = 'gray'

# Function to load image and detect panels
def load_image_and_detect():
    detect_panels_on_image()

# Create the Load Image button
load_button = tk.Button(root, text="Load Image", command=load_image_and_detect)
load_button.pack(pady=10)

detect_video_button = tk.Button(root, text="Load Video", command=detect_panels_on_video)
detect_video_button.pack(pady=10)

# Start the Tkinter main loop
root.mainloop()


No gps information
1/1 [==============================] - 0s 36ms/step
